## Import statements

In [1]:
# Import required packages
import pandas as pd
import numpy as np
import networkx as nx
import random
from scipy.stats import ttest_1samp
from networkx.algorithms.community import modularity
import community.community_louvain as community_louvain
from itertools import combinations
import statistics

random.seed(123)

## Function definitions

In [2]:
# Function to generate random networks
def generate_random_weighted_network_with_expected_degree(degree_seq, original_weights, original_node_labels):
    while True:
        # Generate a random network with the expected degree distribution
        randomized_graph = nx.expected_degree_graph(degree_seq, selfloops=False)

        if nx.is_connected(randomized_graph):
            # Reassign original node labels
            mapping = {old_label: new_label for old_label, new_label in zip(randomized_graph.nodes(), original_node_labels)}
            randomized_graph = nx.relabel_nodes(randomized_graph, mapping)

            # Reassign weights randomly to the edges
            randomized_weights = random.sample(original_weights, len(original_weights))
            for (u, v), weight in zip(randomized_graph.edges(), randomized_weights):
                randomized_graph[u][v]["weight"] = weight
            
            return randomized_graph

In [3]:
# Function to compute various metrics for a graph
def compute_graph_metrics(graph, node_category_dict):
    metrics = {}
    
    # Average degree
    metrics["average_degree"] = sum(dict(graph.degree()).values()) / len(graph)
    
    # Average weighted degree
    metrics["average_weighted_degree"] = sum(dict(graph.degree(weight="weight")).values()) / len(graph)

    # Average path length (only if graph is connected)
    if nx.is_connected(graph):
        metrics["average_path_length"] = nx.average_shortest_path_length(graph)
        metrics["network_diameter"] = nx.diameter(graph)
        
    # Graph density
    metrics["graph_density"] = nx.density(graph)
    
    # Modularity (using Louvain method)
    partition = community_louvain.best_partition(graph)
    communities = {}
    for node, comm_id in partition.items():
        communities.setdefault(comm_id, set()).add(node)
    modularity_communities = list(communities.values())
    metrics["modularity"] = modularity(graph, modularity_communities)
    
    # Average clustering coefficient
    metrics["average_clustering"] = nx.average_clustering(graph)

    # Percent of cross-category edges
    num_cc_edges = 0
    for edge in graph.edges:
        first_node = edge[0]
        second_node = edge[1]
        first_category = node_category_dict[first_node]
        second_category = node_category_dict[second_node]
        if first_category != second_category:
            num_cc_edges += 1
    metrics["percent_cross_category_edges"] = num_cc_edges/len(graph.edges)*100

    return metrics

In [4]:
# Function to perform a t-test; is the difference between the male and female network
#  significantly different from differences in a random population?
def perform_statistical_analysis(original_value_f, original_value_m, random_values):
    original_difference = original_value_f - original_value_m

    random_differences = [rv1 - rv2 for rv1, rv2 in combinations(random_values, 2)]

    t_stat, p_value = ttest_1samp(random_differences, original_difference)
    
    return {
        "original_difference": original_difference,
        "mean_random_difference": np.mean(random_differences),
        "std_random_difference": np.std(random_differences),
        "t_stat": t_stat,
        "p_value": p_value,
    }

## Perform analysis

In [5]:
# Node map
node_categories_path = "/Users/vsriram/Desktop/GxS/ddnComp_personalComputer/traitDescription_diseaseCategories.csv"
node_categories = pd.read_csv(node_categories_path, sep=",")[['Id', 'Disease Category']].set_index('Id')['Disease Category'].to_dict()

In [6]:
# Import male and female networks
f_file_path = "/Users/vsriram/Desktop/GxS/ddnComp_personalComputer/ddnsForDDNComp/ssDDNneg4_femaleBlock_edgeMap.tsv"
f_edges_df = pd.read_csv(f_file_path, sep="\t").iloc[:, :3]
f_edges_df['Weight'] = pd.to_numeric(f_edges_df['Weight'], errors='coerce')

m_file_path = "/Users/vsriram/Desktop/GxS/ddnComp_personalComputer/ddnsForDDNComp/ssDDNneg4_maleBlock_edgeMap.tsv"
m_edges_df = pd.read_csv(m_file_path, sep="\t").iloc[:, :3]
m_edges_df['Weight'] = pd.to_numeric(m_edges_df['Weight'], errors='coerce')

In [7]:
f_node_labels = list(set(f_edges_df['Source']).union(set(f_edges_df['Target'])))
m_node_labels = list(set(m_edges_df['Source']).union(set(m_edges_df['Target'])))

In [8]:
# Create the original graphs
G_f = nx.Graph()
G_f.add_weighted_edges_from(f_edges_df.values)

G_m = nx.Graph()
G_m.add_weighted_edges_from(m_edges_df.values)

In [9]:
# Compute the degree sequences
f_degree_sequence = [d for n, d in G_f.degree()]
m_degree_sequence = [d for n, d in G_m.degree()]

In [10]:
# Extract edge weights
f_original_weights = [d["weight"] for _, _, d in G_f.edges(data=True)]
m_original_weights = [d["weight"] for _, _, d in G_m.edges(data=True)]

In [11]:
# Generate 50 random networks per sex
num_random_networks = 1000

f_random_networks = [
    generate_random_weighted_network_with_expected_degree(f_degree_sequence, f_original_weights, f_node_labels)
    for _ in range(num_random_networks)
]


In [12]:
m_random_networks = [
    generate_random_weighted_network_with_expected_degree(m_degree_sequence, m_original_weights, m_node_labels)
    for _ in range(num_random_networks)
]

random_networks = f_random_networks + m_random_networks

In [13]:
# Compute metrics for the original graphs
f_original_metrics = compute_graph_metrics(G_f, node_categories)
m_original_metrics = compute_graph_metrics(G_m, node_categories)

In [14]:
# Compute metrics for each random network
random_metrics = [compute_graph_metrics(net, node_categories) for net in random_networks]

In [15]:
# Perform t-tests for each metric
results = {}
for metric in f_original_metrics.keys():
    random_values = [rm[metric] for rm in random_metrics]
    results[metric] = perform_statistical_analysis(f_original_metrics[metric], m_original_metrics[metric], random_values)


In [16]:
random_metrics

[{'average_degree': 12.718446601941748,
  'average_weighted_degree': 1372.6019417475727,
  'average_path_length': 2.187321530553969,
  'network_diameter': 4,
  'graph_density': 0.12469065296021321,
  'modularity': 0.6738536361800113,
  'average_clustering': 0.4071671728122718,
  'percent_cross_category_edges': 92.67175572519083},
 {'average_degree': 12.79611650485437,
  'average_weighted_degree': 1333.2427184466019,
  'average_path_length': 2.2381496287835523,
  'network_diameter': 5,
  'graph_density': 0.12545212259661145,
  'modularity': 0.6383892775291848,
  'average_clustering': 0.4132179413200597,
  'percent_cross_category_edges': 93.17147192716236},
 {'average_degree': 11.728155339805825,
  'average_weighted_degree': 1244.2718446601941,
  'average_path_length': 2.286502950694841,
  'network_diameter': 5,
  'graph_density': 0.11498191509613555,
  'modularity': 0.6841710537633592,
  'average_clustering': 0.3730781253290286,
  'percent_cross_category_edges': 92.05298013245033},
 {'a

In [17]:
# Print results
for metric, stats in results.items():
    print(f"{metric.capitalize().replace('_', ' ')}:")
    print(f"  Female Original Value: {f_original_metrics[metric]:.4f}")
    print(f"  Male Original Value: {m_original_metrics[metric]:.4f}")
    print(f"  Random Average: {statistics.mean(rd_metric[metric] for rd_metric in random_metrics):.4f}")
    print(f"  Random Standard Deviation: {statistics.stdev(rd_metric[metric] for rd_metric in random_metrics):.4f}")
    print(f"  T-Statistic: {stats['t_stat']:.4f}" if stats['t_stat'] is not None else "  T-Statistic: N/A")
    print(f"  P-Value: {stats['p_value']:.4f}" if stats['p_value'] is not None else "  P-Value: N/A")
    if stats["p_value"] is not None:
        if stats["p_value"] < 0.05:
            print("  Significant difference detected.")
        else:
            print("  No significant difference detected.")
    print("-" * 50)


Average degree:
  Female Original Value: 13.1262
  Male Original Value: 11.6117
  Random Average: 11.9598
  Random Standard Deviation: 0.7284
  T-Statistic: -1532.6272
  P-Value: 0.0000
  Significant difference detected.
--------------------------------------------------
Average weighted degree:
  Female Original Value: 1391.3592
  Male Original Value: 904.4466
  Random Average: 1106.3587
  Random Standard Deviation: 230.5870
  T-Statistic: -1569.5479
  P-Value: 0.0000
  Significant difference detected.
--------------------------------------------------
Average path length:
  Female Original Value: 2.1611
  Male Original Value: 2.3503
  Random Average: 2.2568
  Random Standard Deviation: 0.0575
  T-Statistic: 3020.4514
  P-Value: 0.0000
  Significant difference detected.
--------------------------------------------------
Network diameter:
  Female Original Value: 4.0000
  Male Original Value: 6.0000
  Random Average: 4.6800
  Random Standard Deviation: 0.5699
  T-Statistic: 3262.6454
 

# Save out clusters from Louvain cluster

In [18]:
random.seed(123)

In [19]:
# Modularity (using Louvain method)
def get_louvain_clusters(graph, node_categories):
    partition = community_louvain.best_partition(graph)
    communities = {}
    for node, comm_id in partition.items():
        communities.setdefault(comm_id, set()).add(node)
    
    # Convert the dictionary into a list of tuples (node, cluster)
    data = [(node, cluster) for cluster, nodes in communities.items() for node in nodes]

    # Create a pandas DataFrame
    df = pd.DataFrame(data, columns=["Node", "Cluster"])
    df['disease_category'] = df['Node'].map(node_categories)
    return df

In [20]:
clusters_f = get_louvain_clusters(G_f, node_categories)
clusters_m = get_louvain_clusters(G_m, node_categories)

In [21]:
clusters_f.to_csv("/Users/vsriram/Desktop/G_f_clusters.csv", index=False)
clusters_m.to_csv("/Users/vsriram/Desktop/G_m_clusters.csv", index=False)